In [1]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
# import seaborn as sns
# from pathlib import Path
import pandas_datareader as web
import requests
from dotenv import load_dotenv
import os
import pprint

In [9]:
p = web.get_data_yahoo(['^GSPC','CL=F','BTC-USD'], '10/31/2017', interval='d')
p = p.iloc[:, [0,1,2]]
p= p.dropna()
p

Attributes    Adj Close                         
Symbols           ^GSPC       CL=F       BTC-USD
Date                                            
2017-10-31  2575.260010  54.380001   6468.399902
2017-11-01  2579.360107  54.299999   6767.310059
2017-11-02  2579.850098  54.540001   7078.500000
2017-11-03  2587.840088  55.639999   7207.759766
2017-11-06  2591.129883  57.349998   7022.759766
...                 ...        ...           ...
2022-11-08  3828.110107  88.910004  18541.271484
2022-11-09  3748.570068  85.830002  15880.780273
2022-11-10  3956.370117  86.470001  17586.771484
2022-11-11  3992.929932  88.959999  17034.292969
2022-11-14  4001.419922  86.070000  16546.943359

[1269 rows x 3 columns]